In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dtutil.sqltools import query_to_dataframe, get_cursor_from_url, list_to_query_list
import pylab
pylab.rcParams['figure.figsize'] = (12,8)

In [65]:
# read in session data
import dtutil.configs as dtc
df = pd.read_parquet(dtc.paths.tlm_sd_path)
df.head()

,id,category,ident,count,session_id,newcnt,opncnt,user_type,serial_num,user_id,user,company,app,added,week,userweek
0,782009,MWOElement,TLIN,24,57910,0,24,customer,53695,686,Cust# 7514,LeGrand,M,2017-01-06 18:10:36,701,686701
1,782008,Command,Tools:SimulateOptimize,1,57910,0,0,customer,53695,686,Cust# 7514,LeGrand,M,2017-01-06 18:10:36,701,686701
2,782007,Command,File:FileLicenseFeatureSetup,1,57910,0,0,customer,53695,686,Cust# 7514,LeGrand,M,2017-01-06 18:10:36,701,686701
3,782006,Command,Project:OptionsProject,1,57910,0,0,customer,53695,686,Cust# 7514,LeGrand,M,2017-01-06 18:10:36,701,686701
4,782005,Command,File:FileProjectSaveAs,1,57910,0,0,customer,53695,686,Cust# 7514,LeGrand,M,2017-01-06 18:10:36,701,686701


In [69]:
dfp = df[(df.user_type == 'pirate') & (df.week >= 800)]
len(dfp)

2286423

In [106]:
dfpk = dfp[(dfp.category == 'PDK') & dfp.ident.apply(lambda x: ("AWR" not in x) and ("Generic" not in x))]
pdk_use = dfpk.groupby(['user_id', 'ident']).id.agg(len)
pdk_use

user_id  ident                        
522      Modelithics Library                3
544      AMP_BLF989                         1
         AMP_BLP9G0722_20G                  2
         AMP_Product                      145
         Antenna PCB Process                1
         Appdata_Files                      3
         Archive                           32
         Balance_LNA                        1
         CREE_G28V5                         1
         CREE_GaN                          27
         Cree_GaN                           3
         DMM_SG13                           3
         EPCOS_CML_SAW2                     3
         FSL_A2I25D025N_Level2_Rev0         2
         FSL_AFT05MS003N_Level1_Rev0        1
         FSL_AFT05MS004N_Level1_Rev0        7
         FSL_AFT09MS007N_Level1_Rev0        1
         FSL_AFT09MS015N_Level1_Rev0        3
         FSL_AFT27S010N_Level2_Rev1        39
         FSL_AFV121KH_Level1_Rev0           1
         FSL_MRF7S24250N_Level1_Rev0     

In [76]:
from dtutil.sqltools import get_cursor_from_url, query_to_dataframe
cur = get_cursor_from_url()

In [103]:
query = """
select distinct ident_user_id, s.user_name, s.machine_name, st.username, st.machine from tlm_session as s
    inner join tlm_start as st on s.start_id = st.id
where ident_user_id in {}
and ((s.user_name != 'None') or (s.machine_name != 'None') or (st.username != '') or (st.machine != ''))
""".format(list_to_query_list(list(dfpk.user_id.unique())))
users = query_to_dataframe(cur, query)

In [104]:
users.head()

,ident_user_id,user_name,machine_name,username,machine
0,522,dektop (Desktop),DESKTOP-CGR3QNL,dektop (Desktop),DESKTOP-CGR3QNL
1,522,None,None,dektop (Desktop),DESKTOP-CGR3QNL
2,544,1,1-ПК,,
3,544,1,1-ПК,1,1-ПК
4,544,12406 (卢诣文),LAPTOP-6KBL6CTM,,


In [107]:
pdk_use.to_excel('/tmp/pdk_use.xlsx')
users.to_excel('/tmp/pdk_users.xlsx')